In [36]:
from flask import Flask,jsonify,Response,make_response,request
from flask_restful import Resource, Api, reqparse
import pandas as pd
import numpy as np
import ast
import json
import time
import datetime

In [37]:
app= Flask(__name__)
api=Api(app)

In [38]:
##experiment
##day
path='Experiments DataSet For Excercise-small.csv'

In [39]:
@app.route('/experiment/results')
def getExperiments():
    #Implemento los filtros con la fecha dada
    day = request.args.get('day')    
    #Implemento los filtros con la fecha dada
    date=datetime.datetime.strptime(day, "%Y-%m-%d %H")
    df= pd.read_csv(path)
    df['timestamp']=pd.to_datetime(df['timestamp']).dt.to_period('H').dt.to_timestamp(freq ='H')
    df=df[df['timestamp']==date]
    if (len(df.index)>0):
    
        ##Armo df con los resultados totales(sin tener en cuenta las variantes ni si compraron o no)
        df_resultados=pd.DataFrame(df['experiments'].str.split(",\s+").apply(lambda x: pd.Series(x).value_counts()).sum().sort_values(ascending=False)).reset_index().rename(columns={"index":"experiment",0:"count"})
        df_resultados['id_variante']=df_resultados['experiment'].str.split('=').str[1]
        df_resultados['experiment']=df_resultados['experiment'].str.split('=').str[0]
        df_resultados=df_resultados[['experiment','id_variante','count']]
        #df_resultados.sort_values(by=['experiment'], ascending=False)
        resultados_total=df_resultados.groupby(by=['experiment'])['count'].sum()
    
    #-------------------------------------------------------------------------#
        #Armo df con los resultados que compraron de los experimentos con sus variantes
        df_buy=df[df['event_name']=='BUY']
        df_filt=df[df['user_id'].isin(df_buy['user_id']) & df['item_id'].isin(df_buy['item_id'])]
        df_filt['experiments']=df_filt['experiments'].map(lambda x: x.lstrip('{').rstrip('}'))
        df_resultados_compra=pd.DataFrame(df_filt['experiments'].str.split(",\s+").apply(lambda x: pd.Series(x).value_counts()).sum()).reset_index().rename(columns={"index":"experiment",0:"count"})
        df_resultados_compra['id_variante']=df_resultados_compra['experiment'].str.split('=').str[1]
        df_resultados_compra['experiment']=df_resultados_compra['experiment'].str.split('=').str[0]
        df_resultados_compra=df_resultados_compra[['experiment','id_variante','count']]
        #df_resultados_compra.sort_values(by=['experiment'], ascending=False)
    
    #--------------------------------------------------------------------------#
        #Obtengo resultados
        experimentos_total=[]
        for id in resultados_total.index:
            experimentFound=(resultados_total[resultados_total.index==id]).to_dict() 
            experimentVariants=df_resultados_compra[df_resultados_compra.experiment==id].to_dict(orient='records')
            if (len(experimentVariants)>0):
                variantes=[{'id':experiment['id_variante'],'number_of_purchases':experiment['count']} for experiment in experimentVariants]
                variante_ganadora = max(variantes, key=lambda x:x['number_of_purchases'])
            experimentos_total.append({id:{
                                        'number_of_participants':experimentFound[id],
                                        'winner':variante_ganadora['id'],
                                        'variantes':variantes
                                        }})
        return jsonify({'results':experimentos_total})
    else:
        response = make_response('experimento no encontrado',404)
        response.mimetype = "text/plain"
        return response

In [40]:
@app.route('/experiment/<path:id>/result')
def getExperiment(id):
    #Implemento los filtros con la fecha y el id dado
    day = request.args.get('day')
    date=datetime.datetime.strptime(day, "%Y-%m-%d %H")
    df= pd.read_csv(path)
    df['timestamp']=pd.to_datetime(df['timestamp']).dt.to_period('H').dt.to_timestamp(freq ='H')
    df=df[df['timestamp']==date]
    if (len(df.index)>0):
    
        ##Armo df con los resultados totales(sin tener en cuenta las variantes ni si compraron o no)
        df_resultados=pd.DataFrame(df['experiments'].str.split(",\s+").apply(lambda x: pd.Series(x).value_counts()).sum().sort_values(ascending=False)).reset_index().rename(columns={"index":"experiment",0:"count"})
        df_resultados['id_variante']=df_resultados['experiment'].str.split('=').str[1]
        df_resultados['experiment']=df_resultados['experiment'].str.split('=').str[0]
        df_resultados=df_resultados[['experiment','id_variante','count']]
        #df_resultados.sort_values(by=['experiment'], ascending=False)
        resultados_total=df_resultados.groupby(by=['experiment'])['count'].sum()
    
    #-------------------------------------------------------------------------#
        #Armo df con los resultados que compraron de los experimentos con sus variantes
        df_buy=df[df['event_name']=='BUY']
        df_filt=df[df['user_id'].isin(df_buy['user_id']) & df['item_id'].isin(df_buy['item_id'])]
        df_filt['experiments']=df_filt['experiments'].map(lambda x: x.lstrip('{').rstrip('}'))
        df_resultados_compra=pd.DataFrame(df_filt['experiments'].str.split(",\s+").apply(lambda x: pd.Series(x).value_counts()).sum()).reset_index().rename(columns={"index":"experiment",0:"count"})
        df_resultados_compra['id_variante']=df_resultados_compra['experiment'].str.split('=').str[1]
        df_resultados_compra['experiment']=df_resultados_compra['experiment'].str.split('=').str[0]
        df_resultados_compra=df_resultados_compra[['experiment','id_variante','count']]
        #df_resultados_compra.sort_values(by=['experiment'], ascending=False)
    
    #--------------------------------------------------------------------------#
        #Obtengo resultados
        experimentFound=(resultados_total[resultados_total.index==id]).to_dict()
        experimentVariants=df_resultados_compra[df_resultados_compra.experiment==id].to_dict(orient='records')
        if (len(experimentVariants)>0):
                variantes=[{'id':experiment['id_variante'],'number_of_purchases':experiment['count']} for experiment in experimentVariants]
                variante_ganadora = max(variantes, key=lambda x:x['number_of_purchases'])
        
        variantes=[{'id':experiment['id_variante'],'number_of_purchases':experiment['count']} for experiment in experimentVariants]
        
        #variante_ganadora = max(variantes, key=lambda x:x['number_of_purchases'])
    #if (len(experimentFound)>0):
        return jsonify({'results':{
                                id:{
                                    'number_of_participants':experimentFound[id],
                                    
                                    'winner':int(variante_ganadora['id']),
                                    
                                    'variantes':variantes
                                    }
                                    }
                                    
                       
                       
                       
                       
                       })
    
    #data=resultados[resultados.index==id].to_dict()
    else:
        response = make_response('experimento no encontrado',404)
        response.mimetype = "text/plain"
        return response
        
    
    

In [41]:
if __name__=='__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
/tmp/ipykernel_7662/1911988144.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filt['experiments']=df_filt['experiments'].map(lambda x: x.lstrip('{').rstrip('}'))
127.0.0.1 - - [03/Dec/2021 09:28:07] "GET /experiment/results?day=2021-08-01%2023 HTTP/1.1" 200 -
/tmp/ipykernel_7662/1500416409.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filt['experiments']=df_filt['experiments'].map(lambda x: x.lstrip('{').rstrip('}'))
127.0.0.1 - 